In [1]:
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from sys import path as sys_path

from astropy.visualization import quantity_support
from sunkit_spex.models.physical.albedo import Albedo
from sunkit_spex.models.physical.nonthermal import ThickTarget
from plotting.plotter import plot_fit_results
from sunkit_spex.models.physical.thermal import ThermalEmission
from sunkit_spex.models.scaling import InverseSquareFluxScaling
from sunkit_spex.models.instrument_response import MatrixModel
from sunkit_spex.spectrum.spectrum import SpectralAxis
from sunkit_spex.data.simulated_data import simulate_square_response_matrix

In [2]:
ph_energies_bin_edges = np.linspace(4,80,100)*u.keV
ph_energies_bin_centers = SpectralAxis._centers_from_edges(ph_energies_bin_edges)

count_energies_bin_edges  = ph_energies_bin_edges
count_energies_bin_centers  = ph_energies_bin_centers

In [ ]:
# Define simulated SRM
srm = simulate_square_response_matrix(ph_energies_bin_centers.size)

# Ceate SRM model
srm_model = MatrixModel(
    matrix=srm, input_axis=SpectralAxis(ph_energies_bin_edges), output_axis=SpectralAxis(count_energies_bin_edges), 
    c=1 * u.ct *u.cm**2 * u.ph**-1, 
    _input_units={"x": u.ph *u.keV**-1 *u.s**-1 * u.cm**-2}, 
    _output_units={"y": u.ct* u.keV**-1 * u.s**-1}
)

# Plot the srm matrix
with quantity_support():
    plt.figure()
    plt.imshow(
        srm_model.matrix,
        origin="lower",
        extent=(
            srm_model.input_axis[0].value,
            srm_model.input_axis[-1].value,
            srm_model.output_axis[0].value,
            srm_model.output_axis[-1].value,
        ),
        norm=LogNorm(),
    )
    plt.ylabel(f"Photon Energies [{srm_model.input_axis.unit}]")
    plt.xlabel(f"Count Energies [{srm_model.output_axis.unit}]")
    plt.title("Simulated SRM")
    plt.show()



In [4]:
# Define models
distance = InverseSquareFluxScaling(1*u.AU)
f_vth = ThermalEmission()
thick = ThickTarget(break_energy=1500*u.keV,low_e_cutoff=20*u.keV)
albedo = Albedo(energy_edges=ph_energies_bin_edges, theta=45*u.deg)

# Create composite models
ph_model= (((f_vth + thick) * distance ) | albedo) 
count_model= (((f_vth + thick) * distance ) | albedo)  | srm_model


In [ ]:
# Evaluate the models
ph_model_eval = ph_model(ph_energies_bin_edges)
count_model_eval = count_model(ph_energies_bin_edges)

In [ ]:
print(ph_model_eval)

In [ ]:
print(count_model_eval)

In [ ]:
ph_with_albedo = (((f_vth + thick) * distance ) | albedo)(ph_energies_bin_edges) 
ph_without_albedo = ((f_vth+thick)*distance)(ph_energies_bin_edges)

ph_albedo = ph_with_albedo - ph_without_albedo

with quantity_support():
    plt.figure()
    plt.stairs(ph_model_eval,ph_energies_bin_edges,baseline=None)
    plt.stairs((f_vth*distance)(ph_energies_bin_edges),ph_energies_bin_edges,baseline=None)
    plt.stairs((thick*distance)(ph_energies_bin_edges),ph_energies_bin_edges,baseline=None)
    plt.stairs(ph_albedo,ph_energies_bin_edges,baseline=None)
    plt.loglog()
    plt.ylim(1e-1,1e6)

In [ ]:
ct_with_albedo = ((((f_vth + thick) * distance ) | albedo)  | srm_model)(ph_energies_bin_edges) 
ct_without_albedo = (((f_vth + thick) * distance )   | srm_model)(ph_energies_bin_edges)

ct_albedo = ct_with_albedo - ct_without_albedo

with quantity_support():
    plt.figure()
    plt.stairs(count_model_eval,count_energies_bin_edges,baseline=None)
    plt.stairs(((f_vth*distance)|srm_model)(ph_energies_bin_edges),count_energies_bin_edges,baseline=None)
    plt.stairs(((thick*distance)|srm_model)(ph_energies_bin_edges),count_energies_bin_edges,baseline=None)
    plt.stairs(ct_albedo,count_energies_bin_edges,baseline=None)
    plt.loglog()
    plt.ylim(1e-1,1e6)